<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/FCN_Experiments/MNIST_uncert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 装载Google硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/FCN_Experiments/'

In [6]:
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 0 --opt HMC --gpu 0 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt HMC --gpu 1 &

# dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'HMC', 'gpu': '0'}
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 2, 'opt': 'HMC', 'gpu': '1'}

# epsilon:  the strength of the adversary, 在测试点周围多大的区域里考虑它的鲁棒性
eps = dict['eps']
# lamada: the probability of seeing the clean data (lambda = 1.0 means no adversarial data considered, lambda = 0.0 means only adversarial data considered)
lam = dict['lam'] 
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob'] 
# the variational inference method
optim = dict['opt']
gpu = dict['gpu']

inference = optim

In [7]:
import BayesKeras
from BayesKeras import PosteriorModel
from BayesKeras import analyzers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

import numpy as np


(F_train, yf_train), (Xf_test, yf_test) = tf.keras.datasets.fashion_mnist.load_data()
F_train = F_train/255.
F_train = F_train.astype("float32").reshape(-1, 28*28)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28* 28)



num_images = 500

model = PosteriorModel(experiment + "%s_FCN_Posterior_%s"%(inference, rob))

epistemic, aleatoric = analyzers.variational_uncertainty(model, X_test[0:num_images])
auc = analyzers.auroc(model, X_test[0:num_images], y_test[0:num_images])

print("ON MNIST")
print("-------------------------------")
print("Epistemic: ",np.mean(epistemic))
print("Aleatoric: ",np.mean(aleatoric))
print("AUC: ", auc)
print("-------------------------------")

epistemic, aleatoric = analyzers.variational_uncertainty(model, F_train[0:num_images])
auc = analyzers.auroc(model, F_train[0:num_images], yf_train[0:num_images])

print("ON FasionMNIST")
print("-------------------------------")
print("Epistemic: ",np.mean(epistemic))
print("Aleatoric: ",np.mean(aleatoric))
print("AUC: ", auc)
print("-------------------------------")
llr = analyzers.likelihood_ratio(model, X_test[0:num_images], F_train[0:num_images])
print("LLR: ", llr)
print("-------------------------------")

meth = analyzers.FGSM
loss = tf.keras.losses.SparseCategoricalCrossentropy()
adv = meth(model, X_test[0:num_images], eps=0.1, loss_fn=loss, num_models=10, direction=y_test[0:num_images])

epistemic, aleatoric = analyzers.variational_uncertainty(model, adv)
auc = analyzers.auroc(model, adv, y_test[0:num_images])
print("ON Adversarial Examples")
print("-------------------------------")
print("Epistemic: ",np.mean(epistemic))
print("Aleatoric: ",np.mean(aleatoric))
print("AUC: ", auc)
print("-------------------------------")
llr = analyzers.likelihood_ratio(model, X_test[0:num_images], adv)
print("LLR: ", llr)
print("-------------------------------")


[INFO] BayesKeras: Attempting to load a sample based posterior
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, None, 512)   401920    
                                                                 
 dense_1 (Dense)             (None, None, None, 10)    5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None
Pred shape:  (35, 500, 10)
ON MNIST
-------------------------------
Epistemic:  -1.1450375e-09
Aleatoric:  0.0010201817
AUC:  0.9786991019558233
-------------------------------
Pred shape:  (35, 500, 10)
ON FasionMNIST
-------------------------------
Epistemic:  -3.2724798e-10
Aleatoric:  0.0011630136
AUC:  0.49646192836920405
-----------